<a href="https://colab.research.google.com/github/LSH0414/Study/blob/master/%EB%89%B4%EC%8A%A4_%EA%B8%B0%EC%82%AC_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 날짜별 마지막 페이지수 정보 가져오기

In [47]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

import time

In [54]:
def getDivDate(date, target):
  if target == 'm':
    result = date.month
  elif target == 'd':
    result = date.day
    
  if result > 9:
    return str(result)
  else:
    return '0' + str(result)

In [15]:
headers = {
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
}

In [204]:
import pickle

start_date = pd.to_datetime('20230201')
end_date = pd.to_datetime('20230317')
search_date = start_date

econ_url = 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=259&sid1=101&date=2023'
before_searching_article = ''

date_article_page = dict()
while True:
  search_month = getDivDate(search_date, 'm')
  search_day = getDivDate(search_date, 'd')
  page = 999
  url = econ_url + search_month + search_day + "&page=" + str(page)

  res = requests.get(url, headers = headers)

  if res.status_code != 200:
    print('Page Article REQUEST ERROR!!')

  soup = BeautifulSoup(res.text, 'html.parser')
  page = soup.select('#main_content > div.paging > strong')[0].text

  date_article_page[search_month + search_day] = page

  if search_date > end_date:
    break
  else:
    search_date += pd.to_timedelta('1D')


with open ('page_data', 'wb') as f:
  pickle.dump(date_article_page, f)


## Scrapy 사용

**저는 colab에서 작업했습니다.**

pwd, cd명령어를 활용해서 시작할 위치를 잡아주시면 됩니다.

- pwd : 현재 위치
- cd [이동할 폴더명] : 위치 이동

In [ ]:
# !pip install scrapy

In [ ]:
# cd 명령어를 사용해서 scrapy를 만들 장소(폴더)로 이동합니다.

In [143]:
# scrapy 초기화(생성) startproject [이름 입력]
# scrapy 프로젝트 만드는 과정입니다. 이름은 자유!
!scrapy startproject navernews

New Scrapy project 'navernews', using template directory '/usr/local/lib/python3.9/dist-packages/scrapy/templates/project', created in:
    /content/navernews

You can start your first spider with:
    cd navernews
    scrapy genspider example example.com


In [146]:
# spider폴더로 이동해야 하므로 cd사용해서 이동해줍니다. cd ./폴더명/폴더명/spiders
cd ./navernews/navernews/spiders

/content/navernews/navernews/spiders


In [148]:
# 크롤링을 진행할 메인 py 파일을 초기화(생성)
#genspider [이름] 도메인 < 이름은 원하시는 이름으로 대신 위에서 생성한 프로젝트 이름과 겹치면 안됩니다. 도메인 초기화는 저도 아직 잘 모르겠네요.. 이걸 해줘야한다고 합니다.
!scrapy genspider navernewsspider news.naver.com

Created spider 'navernewsspider' using template 'basic' in module:
  navernews.spiders.navernewsspider


**여기까지 하시면 크롤링을 위한 밑작업은 끝났습니다. 밑에는 자동으로 생성된 파일들을 수정해서 크롤링하는 과정입니다.**

In [ ]:
# 폴더이름/폴더이름/items.py

"""
items.py는 scrapy 프로젝트를 만들면 기본으로 만들어지는 곳입니다.
크롤링 한 정보를 담아둘 수납공간이라고 생각하시면 됩니다. 
기존에 크롤링했을때 사용한 변수라고 생각하시면 될 것 같아요.
"""

import scrapy


class NavernewsItem(scrapy.Item):
    source = scrapy.Field() # 신문사
    category = scrapy.Field() # 카테고리
    title = scrapy.Field() # 제목
    url = scrapy.Field() # 기사링크
    article = scrapy.Field() # 


In [ ]:
# 폴더이름/폴더이름/settings.py

"""
저는 이부분이 처음에 True로 설정되어 있어서 계속 파싱을 못했습니다. 
ROBOTSTEXT_OBEY 값이 True일 경우 사이트에서 설정한 값을 확인하고 로봇이 접근을 못하게 했다면 접근을 안한다고 하네요.
실제로 위 링크 들어가보면 네이버 뉴스는 로봇 접근을 막아놨습니다. (Disallow : \ <이게 전부 막는다는 뜻이라고 합니다.)
그런데 이걸 False로 바꿔주면 로봇이여도 접근이 가능하다고 합니다. 근데 가급적이면 막아놓은 곳은 사용 안하는게 좋다고 하네요. 자세한건 저도 좀 더 찾아봐야할 것 같아요.
(사이트다마다 url끝에 /robots.txt를 붙여서 들어가서 확인하실 수 있습니다. ex. https://news.naver.com/robots.txt)
"""
ROBOTSTXT_OBEY = False


In [ ]:
# 폴더이름/폴더이름/spiders/크롤링 진행할 메인 파일.py

import scrapy
from navernews.items import NavernewsItem #위에서 사용할 Item의 공간을 미리 만들어놨으니 사용하기 위해서 불러오는 과정
import time

import pickle # 크롤링할 페이지 날짜와, 마지막 페이지 정보를 읽어오기 위한 모듈
import csv # url을 불러오기 위한 모듈

# 맨 처음 날짜와, 마지막 페이지 정보 가져오기
with open ('page_data', 'rb') as r:
  page_data = pickle.load(r)

#네이버 -> 카테고리 들어갔을때 보이는 기사들의 정보를 크롤링하는 class 입니다.
class naverNewsUrlSpider(scrapy.Spider):
    name = "naverNewsUrlSpider"
    allowed_domains = ["news.naver.com"]
    start_urls = ["http://news.naver.com/"]

    # 클래스를 호출할때 자동으로 실행되는 부분입니다.
    def __init__(self, *args, **kargs):
        
        # 탐색할 URL을 저장하는 변수
        self.start_urls = []

        # key값에 '0201'의 형태로 '0317'까지 들어있기 때문에 하나씩 불러와 url을 만들어줍니다.
        for date, end_page in page_data.items():
          url = f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=259&sid1=101&date=2023{date}&page='

          # value값에는 마지막 페이지 정보가 들어있으니 1페이지부터 마지막 페이지까지 반복하면서 탐색할 URL을 탐색 리스트에 추가해줍니다.
          for page in range(1, int(end_page)):
            self.start_urls.append(url + str(page))
        

    # Scrapy로 크롤링을 실행시키면 이 함수가 실행됩니다. 순서상 위 init이 먼저 실행되고 이 함수가 실행되니 탐색 URL에 대한 탐색을 실행하게 됩니다.
    # yield scrapy.Request(url, self.parse) 이 부분은  밑에 있는 parse로 request결과를 보내주는 것이라 생각하시면 됩니다.
    def start_requests(self):
        for url in self.start_urls:
            yield scrapy.Request(url, self.parse)
 
    #  start_requests에서 이 함수를 호출합니다. 여기서 뷰티플숩, 셀레니움에서 하던 원하는 정보 추출작업을 합니다.
    # item=NavernewsItem()이 부분에서 아까 만들어 두었던 공간을 활용합니다.
    # item['title'] = .... 이렇게 작성하면 scrapy에서 정보를 알아서 잘 보관해 줍니다.
    # 추가로 더 크롤링 하고 싶은신 부분이 있다면 items.py에 추가하시고 여기서 그 값을 넣어주시면 됩니다.
    def parse(self, response):
        
        contents = response.xpath('//*[@id="main_content"]/div[2]/ul[1]/li')

        for content in contents:
            item = NavernewsItem()
            item['title'] = content.xpath('dl/dt[2]/a/text()').extract()
            item['source'] = content.xpath('dl/dd/span[2]/text()').extract()
            item['url'] = content.xpath('dl/dt[2]/a/@href').extract()
            item['category'] = '경제'
            yield item

class naverNewsSpider(scrapy.Spider):
    name = "naverNewsSpider"
    
    # 위 클래스에서 url 정보를 csv파일로 저장했으니 불러와서 하나씩 들어가서 기사 전문을 가져오는 작업입니다. 로직은 위에서 설명한 것과 크게 다를게 없습니다.
    def start_requests(self):
      with open('test.csv') as csvfile:
            news_info = csv.DictReader(csvfile)
            for row in news_info:
              if row['url'] == '': # 저는 url을 못가져온 기사들도 목록에 있어서 url이 없으면 넘기도록 했습니다.
                continue
              yield scrapy.Request(row['url'], self.parse_news)

    def parse_news(self, response):
      
      item = NavernewsItem()
      item['title'] = response.xpath('//*[@id="title_area"]/span/text()').extract()
      item['source'] = response.xpath('//*[@id="ct"]/div[1]/div[1]/a/img[1]/@title').extract()
      item['category'] = '경제'
      item['article'] = response.xpath('//*[@id="dic_area"]/text()').extract()

      time.sleep(0.05) # 슬립을 안주면 중간에 크롤링이 막혀서 추가했습니다.
      yield item

In [ ]:
# URL정보를 가져오는 클래스를 호출합니다.
# scrapy crawl [클래스 이름] < 이 부분은 spiders폴더에서 진행했던 py폴더에서 class [이름] 이렇게 작성했던 이름을 써주시면 됩니다.
# [-o '저장할 이름.csv -t csv] 이부분은 csv파일로 저장하기 위해서 추가했습니다.
!scrapy crawl naverNewsUrlSpider -o test.csv -t csv

### 결과물

30초 정도 걸렸습니다.

![img](https://user-images.githubusercontent.com/119479455/226164548-49e3d2a1-527c-463e-9704-8bb048fd6343.png)


In [ ]:
# 위 명령어와 똑같습니다.
!scrapy crawl naverNewsSpider -o test2.csv -t csv

### 결과물

8770개 기사 크롤링하는데 11분 정도 걸렸습니다.

(슬립이 시간이 7분정도 걸려서 오래걸린것 같긴해요...)

![img](https://user-images.githubusercontent.com/119479455/226164572-cc7820a0-cb2a-4214-8b6d-233d83dc9422.png)